## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import re
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation

import pandas as pd


In [2]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [3]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

# see https://www.sqlite.org/lang_select.html for sqlite query syntax
# SELECT (what you want to return) FROM (table)
query_results = convention_cur.execute(
                            '''
                            SELECT text, party
                            FROM conventions
                            ''')

punctuation = set(punctuation) # a set of all punctuation
whitespace = re.compile(r"\s+") # regex to ID whitespace; not necessary because of hashtags
sw = set(stopwords.words("english")) # define the stopwords

# the following is taken from previous Module assignments with minor adjustments
# see https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
def remove_stop(tokens) :
    # modify this function to remove stopwords
    return[w for w in tokens if w not in sw]

def remove_punctuation(text, punct_set=punctuation) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) :
    # we'll have it return i, where i = tokens
    return(i for i in whitespace.split(text))

def prepare(text, pipeline) : 
    tokens = str(text)
    for transform in pipeline : 
        tokens = transform(tokens)    
    return(tokens)

my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

for row in query_results:
    text, party = row
    prep = prepare(text, pipeline=my_pipeline)
    # the following just transforms our tokenized list back into a string of words
    speeches = " ".join(prep)
    convention_data.append([speeches, party])


Let's look at some random entries and see if they look right. 

In [4]:
random.choices(convention_data,k=10)

[['help reform broken criminal justice system joe end private prisons detention centers cash bail school prison pipeline heal soul nation joe biden end hate division trump created stop demonization immigrants coddling white nationalists racist dog whistling religious bigotry ugly attacks women',
  'Democratic'],
 ['oh god truck sinking', 'Democratic'],
 ['six months president trump granted second chance signed first step act law real justice reform brought joy hope freedom thousands well deserving people hollered “hallelujah” faith justice mercy rewarded imagine getting hug loved ones think first step meant much many can’t wait we’re getting started nearly 22 years spent prison wasted god purpose plan life delayed denied destined time pray hear message inspired story compassion lead take action forgotten president donald trump forever grateful god bless god bless president trump god bless america thank',
  'Republican'],
 ['unlike joe biden president trump didn’t choose i’m woman chose

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [5]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [6]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
    # define our dict
    ret_dict = dict()
    # below will tell it to respond with True if word is in the dict
    for i in text.split():
        if i in fw:
            ret_dict[i]=True
    return(ret_dict)

In [7]:
# the three assertion statements were broken up for more convenient troubleshooting
assert(len(feature_words)>0)

In [8]:
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})

In [9]:
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [10]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [11]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [12]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [13]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Within the top 25 most informative features, only 2 of the features are informative in favor of Democrats. 23 of the 25 informative features tell us about the most used words for Republicans, which include many "hot topic" headline words commonly found in trending news articles such as "china", "freedoms", and "enemy". These words also align strongly with Republican ideals such as "enforcement", "defense", and "amendment". Meanwhile, Democratic hot topic words make sense as they align with some of the popular Democratic goals, such as urging for "votes" and "climate" change.


## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [14]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [15]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [16]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

# same as above
punctuation = set(punctuation) # a set of all punctuation
whitespace = re.compile(r"\s+") # regex to ID whitespace; not necessary because of hashtags
sw = set(stopwords.words("english")) # define the stopwords

# the following is taken from previous Module assignments with minor adjustments
# see https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
def remove_stop(tokens) :
    # modify this function to remove stopwords
    return[w for w in tokens if w not in sw]

def remove_punctuation(text, punct_set=punctuation) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) :
    # we'll have it return i, where i = tokens
    return(i for i in whitespace.split(text))

def prepare(text, pipeline) : 
    tokens = str(text)
    for transform in pipeline : 
        tokens = transform(tokens)    
    return(tokens)

my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

for row in results:
    candidate, party, tweet = row
    decode = tweet.decode('utf-8')
    prep = prepare(text=decode, pipeline=my_pipeline)
    # the following just transforms our tokenized list back into a string of words
    prepped_text = " ".join(prep)
    # we don't need the candidate for our classification later
    # eliminating candidate here will remove the following error:
    # ValueError: too many values to unpack (expected 2)
    tweet_data.append([prepped_text, party])

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [17]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [18]:
# let's check
random.choices(tweet_data,k=10)

[['perrynelson thanks', 'Democratic'],
 ['great ag news today → restoring trade access china us beef producers incredible step forward httpstcorpl2dw3dec',
  'Republican'],
 ['icymi bluevirginia jennifer discusses need replace politicians blame mentally ill acts gun violence httpstcoilynevbpqw',
  'Democratic'],
 ['two days womensmarch realdonaldtrumps choice bring back globalgag rule reminds us must fight protectourcare',
  'Democratic'],
 ['today 130pm president barack obama honor recipients 2010 medal freedom ceremony httpfbmevke2veh0',
  'Democratic'],
 ['asked usda inspector general fong recent survey found scientists usda worry could face punishment researching issues relative climate change pollinator health antimicrobial resistance mepolitics watch➡️ httpstcoacisxw8s8a',
  'Democratic'],
 ['happy presidents day realdonaldtrump presidentsday2018 httpstcol8xenamvap',
  'Republican'],
 ['foxx votes require collection unpaid taxes federal employees httptinyurlcom3ajgnh6',
  'Republ

In [19]:

for tweet, party in tweet_data_sample :
    # see https://www.nltk.org/api/nltk.classify.html
    estimate_set = conv_features(tweet, feature_words)
    estimated_party = classifier.classify(estimate_set)
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: let’s make even greater kag 🇺🇸 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: 1hr c

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [20]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimate_set = conv_features(tweet, feature_words)
    estimated_party = classifier.classify(estimate_set)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [21]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3806, 'Democratic': 608}),
             'Democratic': defaultdict(int,
                         {'Republican': 4662, 'Democratic': 926})})

### Reflections

Similar to what was seen earlier in that most informative features belonging to those of Republican political parties, it appears that using those feature words to classify tweets from candidates has a higher accuracy in determining Republican candidates and a low accuracy in determing Democratic candidates. This has occurred because the classifer has overclassified a majority of the candidates as Republican using the feature words. This lines up consistently with our training set of feature words, as it predicted 50% accuracy. By overclassifying candidates as Republican, NB can essentially classify all candidates as Republicans and be correct half of the time.